In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, countDistinct
import pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()

In [12]:
data = pd.DataFrame({'CSNO' : [1, 2, 3], 'REL' : [[5, 2, 7], [1, 9, 3], [4, 8, 6]]})

In [13]:
df = spark.createDataFrame(data)

In [14]:
df.show()

+----+---------+
|CSNO|      REL|
+----+---------+
|   1|[5, 2, 7]|
|   2|[1, 9, 3]|
|   3|[4, 8, 6]|
+----+---------+



### column 삭제 -> drop(column)

In [15]:
df = df.drop(col('REL'))

In [16]:
df.show()

+----+
|CSNO|
+----+
|   1|
|   2|
|   3|
+----+



In [30]:
data = pd.DataFrame({'DATE' : ['2022-01-23', '2022-01-18', '2022-01-24', '2022-01-23', '2022-01-18'],
                    'ID' : ['A', 'B', 'A', 'C', 'A'],
                    'RATING' : [6, 9, 8, 1, 6]})

In [31]:
df = spark.createDataFrame(data)

In [32]:
df.show()

+----------+---+------+
|      DATE| ID|RATING|
+----------+---+------+
|2022-01-23|  A|     6|
|2022-01-18|  B|     9|
|2022-01-24|  A|     8|
|2022-01-23|  C|     1|
|2022-01-18|  A|     6|
+----------+---+------+



### 집계 함수 적용 -> groupBy

In [33]:
df.groupBy('DATE').agg(avg(col("RATING")).alias('avg')).show()

+----------+---+
|      DATE|avg|
+----------+---+
|2022-01-23|3.5|
|2022-01-18|7.5|
|2022-01-24|8.0|
+----------+---+



count 함수는 개수만 세어주는 것이고 중복 제거는 하지 않는다.

In [34]:
df.groupBy('ID').agg(count(col("DATE"))).show()

+---+-----------+
| ID|count(DATE)|
+---+-----------+
|  A|          3|
|  B|          1|
|  C|          1|
+---+-----------+



중복을 제거한 후 count를 하고 싶다면 countDistinct를 해야 한다.

In [37]:
df.groupBy('ID').agg(countDistinct('RATING').alias('no_dup')).show()

+---+------+
| ID|no_dup|
+---+------+
|  B|     1|
|  C|     1|
|  A|     2|
+---+------+



### 정렬 -> orderBy

In [39]:
df.orderBy('DATE').show()

+----------+---+------+
|      DATE| ID|RATING|
+----------+---+------+
|2022-01-18|  A|     6|
|2022-01-18|  B|     9|
|2022-01-23|  A|     6|
|2022-01-23|  C|     1|
|2022-01-24|  A|     8|
+----------+---+------+



### pivot 함수

In [47]:
df.groupBy('DATE').pivot('ID').agg(avg('RATING')).fillna(0).show()

+----------+---+---+---+
|      DATE|  A|  B|  C|
+----------+---+---+---+
|2022-01-18|6.0|9.0|0.0|
|2022-01-23|6.0|0.0|1.0|
|2022-01-24|8.0|0.0|0.0|
+----------+---+---+---+



### collect 함수

In [44]:
df.collect()

[Row(DATE='2022-01-23', ID='A', RATING=6),
 Row(DATE='2022-01-18', ID='B', RATING=9),
 Row(DATE='2022-01-24', ID='A', RATING=8),
 Row(DATE='2022-01-23', ID='C', RATING=1),
 Row(DATE='2022-01-18', ID='A', RATING=6)]

collect()는 해당 데이터의 모든 row를 반환한다.

### describe 함수

In [46]:
df.describe().show()

+-------+----------+----+-----------------+
|summary|      DATE|  ID|           RATING|
+-------+----------+----+-----------------+
|  count|         5|   5|                5|
|   mean|      null|null|              6.0|
| stddev|      null|null|3.082207001484488|
|    min|2022-01-18|   A|                1|
|    max|2022-01-24|   C|                9|
+-------+----------+----+-----------------+

